In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 1404, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 1404 (delta 184), reused 193 (delta 143), pack-reused 1151
Receiving objects: 100% (1404/1404), 78.85 MiB | 12.75 MiB/s, done.
Resolving deltas: 100% (1014/1014), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 15.4 MB/s eta 0:00:00


In [3]:
! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In function ‘__pyx_pf_32MatrixFactorization

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
import scipy.sparse as sps

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv
/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv


In [5]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [6]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_SVDpp_Cython, MatrixFactorization_AsySVD_Cython, MatrixFactorization_WARP_Cython
from Recommenders.MatrixFactorization.PyTorch.MF_MSE_PyTorch import MF_MSE_PyTorch

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

In [7]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [8]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [9]:
config = {
    'tune_parameters_bpr_mf': True,
    'database_path': '/kaggle/working/tuning_mf.db',
    'copy_prev_best_params': False,
    'save_github': True
}

In [10]:
try:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/tuning_mf.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

In [11]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [12]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [13]:
URM_train, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_validation, URM_test = split_train_in_two_percentage_global_sample(URM_test_complete, train_percentage = 0.8)

In [14]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

def objective_function_bpr_mf(trial):
    early_stopping_params = {
        "validation_every_n": 2,
        "stop_on_validation": True,
        "evaluator_object": evaluator,
        "lower_validations_allowed": 10,
        "validation_metric": "MAP",
        "epochs": 500
    }
                          
    recommender_instance = MatrixFactorization_BPR_Cython(URM_train)
    recommender_instance.fit(
        num_factors = trial.suggest_int("num_factors", 1, 500, log=True),
        sgd_mode = trial.suggest_categorical("sgd_mode", ["sgd", "adagrad", "adam", "rmsprop"]),
        batch_size = trial.suggest_categorical("batch_size", [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]),
        user_reg =  trial.suggest_float("user_reg", 1e-6, 1, log=True),
        positive_reg = trial.suggest_float("item_reg", 1e-6, 1e-1, log=True),
        negative_reg = trial.suggest_float("item_reg", 1e-6, 1e-1, log=True),
        learning_rate = trial.suggest_float("learning_rate", 1e-7, 1e-1, log=True),
        WARP_neg_item_attempts = trial.suggest_int("WARP_neg_item_attempts", 1, 100),
        dropout_quota = trial.suggest_float("dropout_quota", 1e-3, 0.7, log=True),
        use_bias=False,
        **early_stopping_params
    )
    
    epochs = recommender_instance.get_early_stopping_final_epochs_dict()["epochs"]
    trial.set_user_attr("epochs", epochs) 
        
    result_df, _ = evaluator.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10, "MAP"]

EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions


In [15]:
if config['tune_parameters_bpr_mf']:
    study = optuna.create_study(direction='maximize', study_name='hyperparameters_tuning_funk_svd',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective_function_bpr_mf, n_trials=15)

[I 2023-12-01 20:38:41,365] Using an existing study with name 'hyperparameters_tuning_funk_svd' instead of creating a new one.


MatrixFactorization_BPR_Cython_Recommender: URM Detected 627 ( 4.8%) users with no interactions.
MatrixFactorization_BPR_Cython_Recommender: URM Detected 247 ( 1.1%) items with no interactions.
MF_BPR: Processed 13312 (100.0%) in 1.12 sec. MSE loss 3.12E-02. Sample per second: 11857
MF_BPR: Epoch 1 of 500. Elapsed time 0.36 sec
MF_BPR: Processed 13312 (100.0%) in 0.43 sec. MSE loss 3.07E-02. Sample per second: 30806
MF_BPR: Validation begins...
EvaluatorHoldout: Processed 10456 (100.0%) in 7.71 sec. Users per second: 1357
MF_BPR: CUTOFF: 10 - PRECISION: 0.0005069, PRECISION_RECALL_MIN_DEN: 0.0008414, RECALL: 0.0006610, MAP: 0.0001624, MAP_MIN_DEN: 0.0003165, MRR: 0.0016028, NDCG: 0.0007195, F1: 0.0005738, HIT_RATE: 0.0049732, ARHR_ALL_HITS: 0.0016135, NOVELTY: 0.0068791, AVERAGE_POPULARITY: 0.0122410, DIVERSITY_MEAN_INTER_LIST: 0.9993289, DIVERSITY_HERFINDAHL: 0.9999233, COVERAGE_ITEM: 0.9186915, COVERAGE_ITEM_HIT: 0.0023717, ITEMS_IN_GT: 0.8004206, COVERAGE_USER: 0.8028256, COVERAGE_U

[I 2023-12-01 20:49:19,473] Trial 155 finished with value: 0.013417916803293557 and parameters: {'num_factors': 152, 'sgd_mode': 'adam', 'batch_size': 1024, 'user_reg': 0.18658364681629766, 'item_reg': 1.5054237047750964e-05, 'learning_rate': 0.002133337337631028, 'WARP_neg_item_attempts': 50, 'dropout_quota': 0.007685574802555596}. Best is trial 150 with value: 0.018714260666010923.


MatrixFactorization_BPR_Cython_Recommender: URM Detected 627 ( 4.8%) users with no interactions.
MatrixFactorization_BPR_Cython_Recommender: URM Detected 247 ( 1.1%) items with no interactions.
MF_BPR: Processed 13312 (100.0%) in 1.22 sec. MSE loss 3.66E-02. Sample per second: 10889
MF_BPR: Epoch 1 of 500. Elapsed time 0.41 sec
MF_BPR: Processed 13312 (100.0%) in 0.59 sec. MSE loss 3.91E-02. Sample per second: 22571
MF_BPR: Validation begins...
EvaluatorHoldout: Processed 10456 (100.0%) in 7.77 sec. Users per second: 1345
MF_BPR: CUTOFF: 10 - PRECISION: 0.0004591, PRECISION_RECALL_MIN_DEN: 0.0006899, RECALL: 0.0004997, MAP: 0.0001486, MAP_MIN_DEN: 0.0002107, MRR: 0.0014856, NDCG: 0.0005823, F1: 0.0004785, HIT_RATE: 0.0045907, ARHR_ALL_HITS: 0.0014856, NOVELTY: 0.0068756, AVERAGE_POPULARITY: 0.0128775, DIVERSITY_MEAN_INTER_LIST: 0.9993690, DIVERSITY_HERFINDAHL: 0.9999273, COVERAGE_ITEM: 0.9342194, COVERAGE_ITEM_HIT: 0.0021479, ITEMS_IN_GT: 0.8004206, COVERAGE_USER: 0.8028256, COVERAGE_U

[I 2023-12-01 21:23:07,545] Trial 156 finished with value: 0.01862419314193414 and parameters: {'num_factors': 182, 'sgd_mode': 'adam', 'batch_size': 1024, 'user_reg': 0.10569780906853109, 'item_reg': 1.230970172373647e-05, 'learning_rate': 0.0030236276626829576, 'WARP_neg_item_attempts': 46, 'dropout_quota': 0.00926629896906296}. Best is trial 150 with value: 0.018714260666010923.


...

[I 2023-12-02 01:43:57,140] Trial 167 finished with value: 0.0001472269282617408 and parameters: {'num_factors': 153, 'sgd_mode': 'adagrad', 'batch_size': 1024, 'user_reg': 0.06705524802966496, 'item_reg': 1.1117066166665619e-05, 'learning_rate': 0.0023345505849170795, 'WARP_neg_item_attempts': 55, 'dropout_quota': 0.006605130795104571}. Best is trial 150 with value: 0.018714260666010923.


MatrixFactorization_BPR_Cython_Recommender: URM Detected 627 ( 4.8%) users with no interactions.
MatrixFactorization_BPR_Cython_Recommender: URM Detected 247 ( 1.1%) items with no interactions.
MF_BPR: Processed 13312 (100.0%) in 0.88 sec. MSE loss 3.63E-02. Sample per second: 15204
MF_BPR: Epoch 1 of 500. Elapsed time 0.42 sec
MF_BPR: Processed 13312 (100.0%) in 1.24 sec. MSE loss 3.59E-02. Sample per second: 10694
MF_BPR: Validation begins...
EvaluatorHoldout: Processed 10456 (100.0%) in 7.72 sec. Users per second: 1354
MF_BPR: CUTOFF: 10 - PRECISION: 0.0004591, PRECISION_RECALL_MIN_DEN: 0.0007192, RECALL: 0.0005385, MAP: 0.0001397, MAP_MIN_DEN: 0.0001998, MRR: 0.0013968, NDCG: 0.0005764, F1: 0.0004956, HIT_RATE: 0.0045907, ARHR_ALL_HITS: 0.0013968, NOVELTY: 0.0068788, AVERAGE_POPULARITY: 0.0121610, DIVERSITY_MEAN_INTER_LIST: 0.9993678, DIVERSITY_HERFINDAHL: 0.9999272, COVERAGE_ITEM: 0.9367253, COVERAGE_ITEM_HIT: 0.0021032, ITEMS_IN_GT: 0.8004206, COVERAGE_USER: 0.8028256, COVERAGE_U

[I 2023-12-02 02:09:14,948] Trial 168 finished with value: 0.013464430690664416 and parameters: {'num_factors': 183, 'sgd_mode': 'adam', 'batch_size': 1024, 'user_reg': 0.09718403381315557, 'item_reg': 1.739277010131567e-05, 'learning_rate': 0.0011775809673015012, 'WARP_neg_item_attempts': 50, 'dropout_quota': 0.009955609788339788}. Best is trial 150 with value: 0.018714260666010923.


MatrixFactorization_BPR_Cython_Recommender: URM Detected 627 ( 4.8%) users with no interactions.
MatrixFactorization_BPR_Cython_Recommender: URM Detected 247 ( 1.1%) items with no interactions.
MF_BPR: Processed 13312 (100.0%) in 0.54 sec. MSE loss 2.57E-02. Sample per second: 24660
MF_BPR: Epoch 1 of 500. Elapsed time 0.28 sec
MF_BPR: Processed 13312 (100.0%) in 0.79 sec. MSE loss 2.53E-02. Sample per second: 16758
MF_BPR: Validation begins...
EvaluatorHoldout: Processed 10456 (100.0%) in 7.43 sec. Users per second: 1407
MF_BPR: CUTOFF: 10 - PRECISION: 0.0003252, PRECISION_RECALL_MIN_DEN: 0.0004681, RECALL: 0.0003246, MAP: 0.0001043, MAP_MIN_DEN: 0.0001647, MRR: 0.0010433, NDCG: 0.0004204, F1: 0.0003249, HIT_RATE: 0.0032517, ARHR_ALL_HITS: 0.0010433, NOVELTY: 0.0068753, AVERAGE_POPULARITY: 0.0123626, DIVERSITY_MEAN_INTER_LIST: 0.9992706, DIVERSITY_HERFINDAHL: 0.9999175, COVERAGE_ITEM: 0.9025373, COVERAGE_ITEM_HIT: 0.0015215, ITEMS_IN_GT: 0.8004206, COVERAGE_USER: 0.8028256, COVERAGE_U

[I 2023-12-02 02:20:04,748] Trial 169 finished with value: 0.013154613892228608 and parameters: {'num_factors': 126, 'sgd_mode': 'adam', 'batch_size': 1024, 'user_reg': 0.08507592945832117, 'item_reg': 1.3091689885869388e-05, 'learning_rate': 0.002488160743014365, 'WARP_neg_item_attempts': 49, 'dropout_quota': 0.007804506148661251}. Best is trial 150 with value: 0.018714260666010923.


In [16]:
if config['tune_parameters_bpr_mf']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [17]:
if config['tune_parameters_bpr_mf']:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

In [18]:
if config['tune_parameters_bpr_mf']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [19]:
# if config['tune_parameters_bpr_mf']:
#     with open('/kaggle/working/best_params_bpr_mf.json', 'w') as params_file:
#         json.dump(study.best_params, params_file)
        
#     if config['save_github']: 
#         upload_file(
#             '/kaggle/working/best_params_bpr_mf.json', 
#             'tuning_results/best_params_bpr_mf.json', 
#             'User KNN tuning results (from kaggle notebook)'
#         )
# elif config['copy_prev_best_params']:
#     shutil.copyfile(
#         '/kaggle/input/hyperparameters-tuning-better/best_params_bpr_mf.json', 
#         '/kaggle/working/best_params_bpr_mf.json'
#     )

In [20]:
if config['save_github'] and config['tune_parameters_bpr_mf']:
    upload_file(
        config['database_path'], 
        'tuning_results/tuning_mf.db',
        'Tuning db updated results (from kaggle notebook)'
    )